In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
test_df = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

# Display Data

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
#firt we will exctracte some useful features from datetime 
train_df['datetime']=pd.to_datetime(train_df['datetime'])
train_df["year"]=train_df.datetime.dt.year
train_df["month"]=train_df.datetime.dt.month
train_df["Day"]=train_df.datetime.dt.day

def hr_func(ts):
    return ts.hour

train_df['hour'] = train_df['datetime'].apply(hr_func)

In [ ]:
train_df.info()

In [ ]:
train_df.hist(figsize= (15, 10))

In [ ]:
train_df.var()

# **Lets Start With  Simple Visualization**

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# sns.countplot(x='season', data=train_df, ax=ax[0,0]);
# sns.countplot(x='holiday', data=train_df, ax=ax[0,1]);
# sns.countplot(x='workingday', data=train_df, ax=ax[1,0]);
# sns.countplot(x='weather', data=train_df, ax=ax[1,1]);

In [ ]:
train_df['registered'].sum()

In [ ]:
train_df['casual'].sum()

In [ ]:
train_df['count'].sum()

In [ ]:
ig,ax=plt.subplots(figsize=(10,5))

sns.barplot(data=train_df,x='season',y='count')
ax.set_title('Rented_bikes Vs season')
plt.show()

In [ ]:
ig,ax=plt.subplots(figsize=(10,5))

sns.barplot(data=train_df,x='Day',y='count')
ax.set_title('Rented_bikes Vs Day')
plt.show()

In [ ]:
ig,ax=plt.subplots(figsize=(10,5))

sns.barplot(data=train_df,x='month',y='count')
ax.set_title('Rented_bikes Vs month')
plt.show()

In [ ]:
ig,ax=plt.subplots(figsize=(10,5))

sns.barplot(data=train_df,x='year',y='count')
ax.set_title('Rented_bikes Vs year')
plt.show()

In [ ]:
train_df[["season", "count"]].groupby(['season'], as_index=False).sum().sort_values(by='count', ascending=False)

In [ ]:
#here we will extract weekday from days and see if weekend will be useful
import datetime as dt
import os
train_df['weekday'] = train_df[['datetime']].apply(lambda x: dt.datetime.strftime(x['datetime'], '%A'), axis=1)


In [ ]:
for i in range(len(train_df['weekday'])):
    if train_df['weekday'][i]=="Sunday" or train_df['weekday'][i]=="Saturday" :
        train_df['weekday'][i] = 0
    else:
        train_df['weekday'][i] = 1
train_df["weekday"] = train_df[["weekday"]].astype(int)

In [ ]:
#from histgram we can see number of rentals increase in the weekend 
plt.hist(train_df.weekday)

* we can see here number of rentals increase at 8-9 and also at 17-20 in working day 
* and when there is no working day we can see number of rentals increase at 10-18

In [ ]:
workday_avg = pd.DataFrame(train_df.groupby(['hour', 'workingday'])['count'].mean()).reset_index()

plt.figure(figsize=(10, 6))
sns.pointplot(data=workday_avg, x='hour', y='count', hue='workingday')

In [ ]:
#draw a seaborn correlation heatmap to detect correlation between features for feature selection process
fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
sns.heatmap(train_df.corr(), annot = True,ax=ax)

# Model Prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X = train_df.drop(['datetime', 'casual', 'registered', 'count', 'Day'], axis=1)
y = train_df['count']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#Root Mean Squared Logarithmic Error

from math import sqrt
def RMSLE(y_pred , y_actual):
    n = y_pred.size 
    RMSLE = sqrt(((np.log(y_pred+1)-np.log(y_actual+1))**2).sum()/n)
    return RMSLE

# Linear Regression

In [ ]:
lr = LinearRegression().fit(X_train, y_train)

train_pred = lr.predict(X_train)
val_pred = lr.predict(X_val)

train_score = RMSLE(train_pred, y_train)
val_score = RMSLE(val_pred ,y_val)

print('Train score:', train_score)
print('validation score:', val_score)

# Random Forest

In [ ]:
#TRy to find the best hyperparameters for RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50,60,70,80,90,100],
              "max_depth": [3, 7, 9 ,10 , 15]}
# kfold = (n_splits=3, shuffle=True, random_state=1)
BestModel= GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1,verbose=3)
BestModel.fit(X_train, y_train)
print(BestModel.best_score_,BestModel.best_estimator_,BestModel.best_params_)

In [ ]:
rf = RandomForestRegressor(max_depth=9, n_estimators=200, random_state=42).fit(X_train, y_train)

tr_pred = rf.predict(X_train)
v_pred = rf.predict(X_val)

tr_score = RMSLE(tr_pred, y_train)
v_score = RMSLE(v_pred ,y_val)

print('validation score:', v_score)
print('Train score:', tr_score)

# Hist Gradient Boosting Regressor

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hist = HistGradientBoostingRegressor(max_depth=15,learning_rate=0.15).fit(X_train , y_train)
y_pred_hist_tr = hist.predict(X_train)
y_pred_hist_ts = hist.predict(X_val)

tr1_score = RMSLE(y_pred_hist_tr, y_train)
v1_score = RMSLE(y_pred_hist_ts ,y_val)

print('validation score:', v1_score)
print('Train score:', tr1_score)

# StackingRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor


estimators = [("Hist",HistGradientBoostingRegressor(max_depth=9,learning_rate=0.15)),("DT",DecisionTreeRegressor(max_depth=5,random_state=42)),("Linear",Ridge(alpha=0.9)),("Extra",ExtraTreesRegressor(max_depth=15,n_estimators=50, random_state=39)),("ran",RandomForestRegressor( n_estimators=70,max_depth=15, random_state=39))]

reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor( n_estimators=60,max_depth=9, random_state=42))

reg.fit(X_train , y_train)
#


y_pred_reg_tr = reg.predict(X_train)
y_pred_reg_ts = reg.predict(X_val)


R_reg= RMSLE(y_pred_reg_ts , y_val)
R1_reg = RMSLE(y_pred_reg_tr , y_train)

print("RMSLE of Random Forest Regressor :",R_reg)
print("RMSLE of what model learn :",R1_reg)


# Test data

In [ ]:
test_df['datetime']=pd.to_datetime(test_df['datetime'])
test_df["year"]=test_df.datetime.dt.year
test_df["month"]=test_df.datetime.dt.month
test_df["Day"]=test_df.datetime.dt.day

def hr_func(ts):
    return ts.hour

test_df['hour'] = test_df['datetime'].apply(hr_func)

In [ ]:
test_df['weekday'] = test_df[['datetime']].apply(lambda x: dt.datetime.strftime(x['datetime'], '%A'), axis=1)

In [ ]:
for i in range(len(test_df['weekday'])):
    if test_df['weekday'][i]=="Sunday" or test_df['weekday'][i]=="Saturday" :
        test_df['weekday'][i] = 0
    else:
        test_df['weekday'][i] = 1
        
test_df["weekday"] = test_df[["weekday"]].astype(int)

In [ ]:
test_df = test_df.drop(['datetime', 'Day'], axis=1)

In [ ]:
test_df

# Submission

In [ ]:
y_test_predicted = reg.predict(test_df)

In [ ]:

test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

submission_df = pd.DataFrame({
        "datetime": test["datetime"],
        "count": y_test_predicted
    })

In [ ]:
submission_df.to_csv("submission.csv", index = False)

In [ ]:
submission_df